In [1]:
!pip install assemblyai

import assemblyai as aai
import os

In [2]:
aai.settings.api_key = "01754830854649d0915488dc8964f835"


In [3]:
config = aai.TranscriptionConfig(
    speaker_labels=True,
    sentiment_analysis=True,
    speech_model=aai.SpeechModel.best
)

In [4]:
local_path = os.path.expanduser("~/Desktop/EY_Project/Sample Order Taking Customer Support Philippines [TubeRipper.com].mp3")

config = aai.TranscriptionConfig(
    speaker_labels=True,
    sentiment_analysis=True,
    speech_model=aai.SpeechModel.best
)

transcriber = aai.Transcriber(config=config)
transcript = transcriber.transcribe(local_path)

if transcript.status == "error":
    raise RuntimeError(f"Transcription failed: {transcript.error}")

In [5]:
for utt in transcript.utterances:
    print(f"[Speaker {utt.speaker}] {utt.text}")

[Speaker B] Thank you for calling Martha's Florist. How mess.
[Speaker A] Hello. I'd like to order flowers and I think you have what I'm looking for.
[Speaker B] I'd be happy to take care of your order. May I have your name, please?
[Speaker A] Randall Thomas.
[Speaker B] Randall Thomas. Can you spell that for me?
[Speaker A] Randall. R, A, N, D, A, L, L. Thomas. T, H, O, M, A, N. Thank.
[Speaker B] You for that information. Randall may have your home or office number, Area code first area code 409.
[Speaker A] Then 866-5088.
[Speaker B] That's 409-866-5088. Do you have a fax number or email address? My email is randall.thomasmail.com randall.thomasmail.Com may have your shipping address.
[Speaker A] 6800.
[Speaker B] Okay.
[Speaker A] Gladys Avenue, Beaumont, Texas. Zip code is 77706.
[Speaker B] Gladys Avenue, BeaumonT, Texas. Zip code 77706. Thank you for the information. What products were you interested in purchasing?
[Speaker A] Red roses. Probably a dozen.
[Speaker B] One dozen 

In [7]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')

analyzer = SentimentIntensityAnalyzer()

results = []
for i, utt in enumerate(transcript.utterances):
    line = utt.text.strip()
    speaker = utt.speaker
    scores = analyzer.polarity_scores(line)
    sentiment = (
        "Positive" if scores['compound'] > 0.05 else
        "Negative" if scores['compound'] < -0.05 else
        "Neutral"
    )
    results.append({
        "Line": i + 1,
        "Speaker": speaker,
        "Text": line,
        "Sentiment": sentiment,
        "Compound Score": scores['compound']
    })

df = pd.DataFrame(results)

overall_sentiment = df['Sentiment'].value_counts().idxmax()

summary_row = {
    "Line": "Overall",
    "Speaker": "",
    "Text": "Conversation Summary",
    "Sentiment": overall_sentiment,
    "Compound Score": df["Compound Score"].mean()  
}
df = pd.concat([df, pd.DataFrame([summary_row])], ignore_index=True)

df.to_excel("transcript1_sentiment.xlsx", index=False)

df.head()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/aramishra/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Line,Speaker,Text,Sentiment,Compound Score
0,1,B,Thank you for calling Martha's Florist. How mess.,Neutral,0.0000
1,2,A,Hello. I'd like to order flowers and I think y...,Positive,0.3612
2,3,B,I'd be happy to take care of your order. May I...,Positive,0.8481
3,4,A,Randall Thomas.,Neutral,0.0000
4,5,B,Randall Thomas. Can you spell that for me?,Neutral,0.0000
